In [ ]:
import matplotlib.pyplot as plt
import torch

from flow_matching_trainer import FlowMatchingModel

In [ ]:
flow_match_model = FlowMatchingModel.load_from_checkpoint(
    "/home/arry_iang/workspace/playground/flow_matching_toy/lightning_logs/version_0/checkpoints/epoch=9-step=37500.ckpt"
)
flow_match_model.to("cuda")
flow_match_model.eval()


In [ ]:
NUM_SAMPLES = 1
NUM_STEPS = 100
GUIDANCE_SCALE = 4

# 0	T-shirt/top, 1 Trouser, 2 Pullover, 3 Dress, 4 Coat, 5 Sandal, 6 Shirt, 7 Sneaker, 8 Bag, 9 Ankle boot
CONDITION = 9
NULL_CONDITION = 10
with torch.no_grad():
    x = torch.randn((NUM_SAMPLES, 1, 32, 32), device="cuda")
    for t in torch.linspace(0, 1, NUM_STEPS, device="cuda"):
        t = t.expand((NUM_SAMPLES,))
        condition = torch.tensor((CONDITION,), dtype=int, device="cuda")
        null_condition = torch.tensor((NULL_CONDITION,), dtype=int, device="cuda")
        flow = (1 - GUIDANCE_SCALE) * flow_match_model(x, null_condition, t) + GUIDANCE_SCALE * flow_match_model(x, condition, t)
        x += 1 / NUM_STEPS * flow

plt.imshow(x.cpu().detach()[0].permute(1, 2, 0))